# IHLT Lab 9: Coreferences

**Author:** *Zachary Parent ([zachary.parent](mailto:zachary.parent@estudiantat.upc.edu))*

### 2024-12-02

**Instructions:**

- Consider the first paragraph in Alice's Adventures in Wonderland, by Lewis Carroll:
  > Alice was beginning to get very tired of sitting by her sister on the bank, 
  > and of having nothing to do: once or twice she had peeped into the book her 
  > sister was reading, but it had no pictures or conversations in it, 'and what is the use of a book,' thought Alice 'without pictures or conversations?'
- It can be downloaded from:
  - http://www.gutenberg.org/files/11/11-0.txt
- Apply the spaCy coreference solver to the previous paragraph.
- Show the coreference chains.
- What do you think about them? Justify your answer.

In [1]:
import spacy

In [2]:
nlp_exp = spacy.load("en_coreference_web_trf")

nlp_coref = spacy.load('en_core_web_lg')
nlp_coref.add_pipe('coreferee')


/Users/zachparent/src/MAI/ihlt/lab9/.venv-spacy/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/zachparent/src/MAI/ihlt/lab9/.venv-spacy/lib/python3.9/site-packages/spacy_transformers/layers/hf_shim.py:118: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allo

In [3]:
paragraph = """Alice was beginning to get very tired of sitting by her sister on the bank, 
  and of having nothing to do: once or twice she had peeped into the book her 
  sister was reading, but it had no pictures or conversations in it,
  'and what is the use of a book,' thought Alice 'without pictures or conversations?'"""

In [4]:
doc_exp = nlp_exp(paragraph)

print(doc_exp.spans)

{'coref_clusters_1': [Alice, her, she, her, Alice], 'coref_clusters_2': [her sister, her 
  sister], 'coref_clusters_3': [the book her 
  sister was reading, it, it]}


/Users/zachparent/src/MAI/ihlt/lab9/.venv-spacy/lib/python3.9/site-packages/thinc/shims/pytorch.py:114: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(self._mixed_precision):


In [5]:
doc_coref = nlp_coref(paragraph)

doc_coref._.coref_chains.print()

0: Alice(0), her(10), she(27), her(33), Alice(62)
1: bank(14), it(47)
2: book(32), it(40)


# Analysis

## Spacy's experimental coreference solver

The coreference chains produced by spaCy's experimental coreference solver are accurate in this case. It is worth being skeptical, because they can sometimes produce incorrect results, so we went item by item to check the results. Below are the annotations from the coreference chains:

> Alice[`1`] was beginning to get very tired of sitting by [her[`1`] sister][`2`] on the bank, 
>  and of having nothing to do: once or twice she[`1`] had peeped into [the book [her[`1`] 
>  sister][`2`] was reading][`3`], but it[`3`] had no pictures or conversations in it[`3`],
>  'and what is the use of a book,' thought Alice[`1`] 'without pictures or conversations?'

These seem to be sensible coreferences, resolving to Alice (`1`), her sister (`2`), and the book (`3`). These coreferences remain consistent throughout the paragraph. The coreferences do not miss any anaphoric references, and the antecedents are correct.

## Coreferee

> Alice[`1`] was beginning to get very tired of sitting by her[`1`] sister on the bank[`2`], 
>  and of having nothing to do: once or twice she[`1`] had peeped into the book[3] her[`1`] 
>  sister was reading, but it[`3`] had no pictures or conversations in it[`2`],
>  'and what is the use of a book,' thought Alice[`1`] 'without pictures or conversations?'

The coreference chains produced by Coreferee are not as accurate. The coreference chains are not as consistent throughout the paragraph, and the antecedents are not always correct. For example, the coreference chains resolve "it" to "bank" in the first sentence, but "it" to "book" in the second sentence. This is an error. It also does not keep track of the sister, which is an error.